In [1]:
#!/usr/bin/env python3
"""
Neven Caplar 
Last updated: 2023-11-30

Goals: 
Fit the data

Each Section can/should run independently,
only these initial imports should be shared among all sections

Open questions:
None at the moment
"""

import os

import numpy as np
import pandas as pd
import pyarrow as pa

# from scipy.spatial import KDTree
import matplotlib.pyplot as plt

import JaxPeriodDrwFit


from tape.ensemble import Ensemble
from tape.utils import ColumnMapper

from tqdm import tqdm

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
import dask
# many workers
# dask.config.set(scheduler='threads') 

dask.config.set({"temporary-directory" :'/epyc/ssd/users/ncaplar/tmp'})

# does not work
# from multiprocessing.pool import ThreadPool
# dask.config.set(pool=ThreadPool(20))

# one worker
# dask.config.set(scheduler='processes')  
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8, threads_per_worker=1)
client = Client(cluster)
# cluster.adapt(minimum=10, maximum=40) 

/astro/users/ncaplar/.conda/envs/tape_static/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37890 instead
  warnings.warn(


In [3]:
dask.config.get("temporary-directory")

'/epyc/ssd/users/ncaplar/tmp'

In [4]:

ens = Ensemble(client = client)  # initialize an ensemble object
ens.client_info()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:37890/status,
Dashboard: http://127.0.0.1:37890/status,Workers: 8
Total threads: 8,Total memory: 251.68 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41717,Workers: 8
Dashboard: http://127.0.0.1:37890/status,Total threads: 8
Started: Just now,Total memory: 251.68 GiB
Comm: tcp://127.0.0.1:43336,Total threads: 1
Dashboard: http://127.0.0.1:38065/status,Memory: 31.46 GiB
Nanny: tcp://127.0.0.1:41963,


In [5]:
dask.config.get("temporary-directory")

'/epyc/ssd/users/ncaplar/tmp'

In [ ]:
# Setup base directory for saving output files
username= "ncaplar"
basedir = f"/astro/users/{username}/data/"

In [6]:
# Setup base directory for saving output files
username= "ncaplar"
basedir = f"/astro/users/{username}/data/"

# ZTF

## ZTF - Loading original data 

In [ ]:
# TODO - move to shuffling notebook

In [ ]:
rel_path = "/data3/epyc/data3/hipscat/catalogs/ztf_dr14_x_agns_source"

from pyarrow import parquet
# parquet.read_schema(f"{rel_path}/Norder=8/Dir=210000/Npix=217286.parquet", memory_map=True)

parquet.read_schema("/epyc/projects3/sean_hipscat/agns_x_ztf_source/Norder=8/Dir=210000/Npix=217286.parquet", memory_map=True)

In [ ]:
"""
colmap = ColumnMapper(id_col="_hipscat_index",
                      time_col="mjd_ztf_source",
                      flux_col="mag_ztf_source",
                      err_col="magerr_ztf_source",
                      band_col="band_ztf_source")
ens.from_parquet(source_file="/epyc/projects3/sean_hipscat/agns_x_ztf_source/Norder*/Dir*/Npix*.parquet",
                 #object_file=datapath+"object/*.parquet",
                 column_mapper=colmap,
                 partition_size="1000MB")
ens._source.reset_index().set_index("_hipscat_index").to_parquet("/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big/")
"""


## ZTF - Repartitioned dataset / partitioning and fitting and shuffling

In [ ]:
# TODO - move the shuffling part to the shuffling notebook

In [ ]:
import glob

directory_path = '/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big/'
file_pattern = 'part.25*.parquet'

matching_files = glob.glob(f'{directory_path}{file_pattern}')

for file_path in matching_files:
    print(file_path)

In [ ]:
names = []
for i in range(250,254):
    name = f"/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big/part.{i}.parquet"
    names.append(name)
    
names

In [ ]:
colmap = ColumnMapper(id_col="_hipscat_index",
                      time_col="mjd_ztf_source",
                      flux_col="mag_ztf_source",
                      err_col="magerr_ztf_source",
                      band_col="band_ztf_source")
ens.from_parquet(source_file=names,
                 #object_file=datapath+"object/*.parquet",
                 column_mapper=colmap,
                 sorted=True)

### load into memory

In [ ]:
ens_object = ens._object.compute()
ens_source = ens._source.compute()

In [ ]:
# in GB
ens_source.memory_usage(deep=True).sum()/1e9

In [ ]:
len(ens_object)

In [ ]:
ens_source['SDSS_NAME_dr16q_constant'].nunique()

### investigating the sample and fit 

In [ ]:
ens.check_lightcurve_cohesion()

In [ ]:
ens.query("band_ztf_source == 'g'", table="source").prune(50)
ens._lazy_sync_tables(table="object")
ens.calc_nobs(temporary=False)

data = ens.compute('object')['nobs_total'].values.astype(int)

bin_edges = range(0, 321 + 21, 20) 
plt.hist(data, bins=bin_edges, edgecolor='k')

In [ ]:
def count_rows(partition):
    return len(partition)

# Let us try the same thing again
n_sources_per_div = ens._source.map_partitions(count_rows, meta=int).compute()

print("Number of rows in each partition:", n_sources_per_div)

In [ ]:
# 18 minutes, 95.17 GB used
JaxPeriodDrwFit_instance = JaxPeriodDrwFit.JaxPeriodDrwFit()
res_tsp_drw = ens.batch(JaxPeriodDrwFit_instance.optimize_map_drw, 'mjd_ztf_source', "mag_ztf_source", "magerr_ztf_source",
                compute=True, meta=None, n_init=100)

## ZTF - reshuffled and fit 

In [7]:
import glob

directory_path = '/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big_shuffled_test/'
file_pattern = 'part.*.parquet'

matching_files = glob.glob(f'{directory_path}{file_pattern}')

for file_path in matching_files:
    print(file_path)

/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big_shuffled_test/part.3.parquet
/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big_shuffled_test/part.2.parquet
/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big_shuffled_test/part.1.parquet
/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big_shuffled_test/part.0.parquet


In [8]:
test_pd = pd.read_parquet(matching_files[0])
test_pd


,count,_hipscat_index,SDSS_NAME_dr16q_constant,PLATE_dr16q_constant,MJD_dr16q_constant,FIBERID_dr16q_constant,RA_dr16q_constant,DEC_dr16q_constant,OBJID_dr16q_constant,IF_BOSS_SDSS_dr16q_constant,...,magerr_ztf_source,mjd_ztf_source,rcID_ztf_source,band_ztf_source,Norder_ztf_source,Dir_ztf_source,Npix_ztf_source,Norder,Dir,provenance
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
0,298.0,7343808069173772288,135628.50+045054.4,856,52339,458,209.118759,4.848461,0856-52339-0458,SDSS,...,0.050652,59274.44767,46,g,6,20000,26090,6,20000,survey_1
1,298.0,7416242961983209472,122749.00+053201.9,2880,54509,581,186.954179,5.533870,2880-54509-0581,SDSS,...,0.079153,58956.25185,39,g,6,20000,26347,6,20000,survey_1
2,298.0,7416242961983209472,122749.00+053201.9,2880,54509,581,186.954179,5.533870,2880-54509-0581,SDSS,...,0.083438,58955.30475,39,g,6,20000,26347,6,20000,survey_1
3,298.0,7416242961983209472,122749.00+053201.9,2880,54509,581,186.954179,5.533870,2880-54509-0581,SDSS,...,0.078276,58955.30152,39,g,6,20000,26347,6,20000,survey_1
4,298.0,7416242961983209472,122749.00+053201.9,2880,54509,581,186.954179,5.533870,2880-54509-0581,SDSS,...,0.069537,58944.25345,39,g,6,20000,26347,6,20000,survey_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638022,1012.0,7488300675512664064,131254.92+151528.7,1773,53112,337,198.228874,15.257989,1773-53112-0337,SDSS,...,0.074948,59639.40354,60,g,6,20000,26603,6,20000,survey_1
638023,1012.0,7488300675512664064,131254.92+151528.7,1773,53112,337,198.228874,15.257989,1773-53112-0337,SDSS,...,0.067980,59625.38157,60,g,6,20000,26603,6,20000,survey_1
638024,1012.0,7488300675512664064,131254.92+151528.7,1773,53112,337,198.228874,15.257989,1773-53112-0337,SDSS,...,0.078997,59625.36090,3,g,6,20000,26603,6,20000,survey_1


In [9]:
names = []
for i in range(0,4):
    name = f"/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big_shuffled_test/part.{i}.parquet"
    names.append(name)
    
names

['/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big_shuffled_test/part.0.parquet',
 '/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big_shuffled_test/part.1.parquet',
 '/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big_shuffled_test/part.2.parquet',
 '/astro/store/epyc3/data3/hipscat/catalogs/ztf_dr14_x_agns_source_repar_big_shuffled_test/part.3.parquet']

In [10]:
colmap = ColumnMapper(id_col="count",
                      time_col="mjd_ztf_source",
                      flux_col="mag_ztf_source",
                      err_col="magerr_ztf_source",
                      band_col="band_ztf_source")
ens.from_parquet(source_file=names,
                 #object_file=datapath+"object/*.parquet",
                 column_mapper=colmap,
                 sorted=True)

In [12]:
ens._object

""
npartitions=4
1.0
146.0
204.0
298.0
1012.0


In [13]:
ens_c_ix = ens._source._hipscat_index.compute()

In [15]:
len(ens_c_ix)

2494598

In [16]:
n_count = np.sort(np.unique(ens_c_ix.values, return_counts=True))[1].astype(int)
n_count

array([   1,    1,    1, ...,  898,  929, 1012])

In [17]:
len(n_count)

16923

In [ ]:
# custom divisions in the middle
n_count[np.array([len(n_count)/4, 2* len(n_count)/4, 3* len(n_count)/4]).astype(int)]


In [ ]:
ens._source.divisions

In [ ]:
custom_divisions = (1.0, 54.0, 135.0, 207.0, 1012.0)
ens._source = ens._source.repartition(divisions=custom_divisions)
ens._source.divisions


In [ ]:
c1 = np.unique(ens._source.get_partition(0).compute()['_hipscat_index'].values, return_counts=True)  
c3 = np.unique(ens._source.get_partition(3).compute()['_hipscat_index'].values, return_counts=True)  

In [ ]:
bin_edges = range(0, 721 + 21, 20) 

plt.figure(figsize=(12, 5))

plt.subplot(121 )
plt.hist(c1[1], bins=bin_edges, edgecolor='k')
plt.ylim(0, 5000)

plt.subplot(122 )
plt.hist(c3[1], bins=bin_edges, edgecolor='k')
plt.ylim(0, 5000)

In [ ]:
# 13.5 minutes, 22.84 GB used without repartitioning

JaxPeriodDrwFit_instance = JaxPeriodDrwFit.JaxPeriodDrwFit()
res_tsp_drw = ens.batch(JaxPeriodDrwFit_instance.optimize_map_drw, 'mjd_ztf_source', "mag_ztf_source", "magerr_ztf_source",
                compute=True, meta=None, n_init=100)